In [1]:
import numpy as np
import cv2
import glob
import os
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt

## Focus assesment algorithm

In [24]:
def sobel_gradmap(img):
    grad_x = cv2.Sobel(img, cv2.CV_16S, 1, 0)
    grad_y = cv2.Sobel(img, cv2.CV_16S, 0, 1)
    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)
    grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    return grad

def features_focus(img):
    rows,cols,_ = img.shape
    img_gs = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
    O = sobel_gradmap(img_gs)
    img_lp_3 = cv2.blur(img_gs,(3,3))
    L1 = sobel_gradmap(img_lp_3)
    img_lp_5 = cv2.blur(img_gs,(5,5))
    L2 = sobel_gradmap(img_lp_5)
    # Compute focus measures
    FM1 = cv2.sumElems(O)/(rows*cols)
    FM2 = FM1 - cv2.sumElems(L1)/(rows*cols)
    FM3 = cv2.sumElems(L1)/(rows*cols) - cv2.sumElems(L2)[0]/(rows*cols)
    return img_gs, img_lp_3, img_lp_5, FM1, FM2, FM3

In [20]:
def show_res_focus(img):
    img = cv2.imread(img)
    im1, im2, im3, fm1, fm2, fm3 = features_focus(img)
    im1 = cv2.cvtColor(im1, cv2.COLOR_GRAY2RGB)
    im2 = cv2.cvtColor(im2, cv2.COLOR_GRAY2RGB)
    im3 = cv2.cvtColor(im3, cv2.COLOR_GRAY2RGB)
    fig = plt.figure(figsize=(12, 10))
    fig.add_subplot(1, 3, 1)
    plt.imshow(im1) 
    plt.axis('off') 
    plt.title("Původní obraz")
    fig.add_subplot(1, 3, 2)
    plt.imshow(im2)
    plt.axis('off') 
    plt.title("2")
    fig.add_subplot(1, 3, 3)
    plt.imshow(im3)
    plt.axis('off')
    plt.title("3")
    print(fm1)
    print(fm2)
    print(fm3)

# def dataset_results_focus(path):
    #TODO

In [ ]:
show_res_focus("./STRaDeSetB_preprocessed/001_L1_2577.JPG")

In [ ]:
show_res_focus("./STRaDeSetB_preprocessed/002_R2_2569.JPG")

In [ ]:
show_res_focus("./STRaDeSetB_preprocessed/Failure_cases/041_F1_3058.JPG")

In [ ]:
show_res_focus("./STRaDeSetB_preprocessed/Failure_cases/031_F2_2940.JPG")

## Brightness assesment algorithm

In [ ]:
def create_colormap(path):
    processed = 0
    for img_path in glob.iglob(f'{path}/*.JPG'):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        # [0, 1] - hue and saturation
        hist = cv2.calcHist([img], [0, 1], None, [180, 256], [0, 180, 0, 256])
        cv2.normalize(hist,hist,0,255,cv.NORM_MINMAX)
        avg += hist
        processed += 1
    avg /= processed # normalization to one common histogram
    return avg
    
def features_brightness(img):
    cm_light = create_colormap("./data/light")
    cm_dark = create_colormap("./data/dark")
    cm_normal = create_colormap("./data/normal")
    rows,cols,_ = img.shape
    bp_bright = cv.calcBackProject([img], [0, 1], cm_light, [0, 180, 0, 256], 1)
    bp_dark = cv.calcBackProject([img], [0, 1], cm_dark, [0, 180, 0, 256], 1)
    bp_normal = cv.calcBackProject([img], [0, 1], cm_normal, [0, 180, 0, 256], 1)
    CM1 = cv2.sumElems(bp_bright)/(rows*cols)
    CM2 = cv2.sumElems(bp_dark)/(rows*cols)
    CM3 = cv2.sumElems(bp_normal)/(rows*cols)
    return CM1, CM2, CM3

In [ ]:
# def show_res_brightness(img):
    #TODO
    
# def dataset_results_brightness(path):
    #TODO
